# raw data에서 Patent 변수 생성에 필요한 info만 걸러내기

### raw data(patent)에서 필요한 정보

1. patent id (이미 완료)
2. assignee (이미 완료)
3. patent date (나중에 year로 변환)
4. patent class
5. citation edge list

### 1. patent year / class

__patent granted year 정보가 없음...=> 교수님과 이야기 필요__  
    대안: citation에 date 정보 (year, month) 있음  
    문제: citation 0인 patent는 date 확인 불가  
    일단 class만 만들고 citation 만들 때 year까지 집어넣자 

__교수님과 이야기 완료__   
class: __subclass까지 다 뽑기__   
date: __raw data 새로 받아서__ 뽑기

### 1.1 main class 뽑기

In [1]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent")

In [2]:
zf_patent1 = zipfile.ZipFile('harvard_vehicle_patent1.zip') 
csv_patent1 = open('harvard_vehicle_patent1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header=csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('harvard_vehicle_patent2.zip') 
csv_patent2 = zf_patent2.open('harvard_vehicle_patent2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)
csv_patent_reader2.next()

['',
 'patent_id_0',
 'project_id',
 'uuid_0',
 'patent_id_0',
 'date_0',
 'kind_0',
 'number_0',
 'country_0',
 'category_0',
 'sequence_0',
 'uuid_1',
 'patent_id_1',
 'date_1',
 'kind_1',
 'number_1',
 'country_1',
 'category_1',
 'sequence_1',
 'uuid_2',
 'patent_id_2',
 'date_2',
 'kind_2',
 'number_2',
 'country_2',
 'category_2',
 'sequence_2',
 'uuid_3',
 'patent_id_3',
 'date_3',
 'kind_3',
 'number_3',
 'country_3',
 'category_3',
 'sequence_3',
 'uuid_4',
 'patent_id_4',
 'date_4',
 'kind_4',
 'number_4',
 'country_4',
 'category_4',
 'sequence_4',
 'uuid_5',
 'patent_id_5',
 'date_5',
 'kind_5',
 'number_5',
 'country_5',
 'category_5',
 'sequence_5',
 'uuid_6',
 'patent_id_6',
 'date_6',
 'kind_6',
 'number_6',
 'country_6',
 'category_6',
 'sequence_6',
 'uuid_7',
 'patent_id_7',
 'date_7',
 'kind_7',
 'number_7',
 'country_7',
 'category_7',
 'sequence_7',
 'uuid_8',
 'patent_id_8',
 'date_8',
 'kind_8',
 'number_8',
 'country_8',
 'category_8',
 'sequence_8',
 'uuid_9',

In [4]:
mainclass_col = []
for j in range(62457,63354):
    if 'mainclass' in header[j]:   
        mainclass_col.append(j)

In [5]:
with open('veh_pid_class.csv','wb') as myfile:
    myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    
    for line in csv_patent_reader1:
        row = []
        row.append(line[0])
        row.append(line[1])
        for i in mainclass_col:
            if line[i] == '':
                break
            else:
                row.append(line[i])
        myfile_writer.writerow(row)
        
    for line in csv_patent_reader2:
        row = []
        row.append(line[0])
        row.append(line[1])
        for i in mainclass_col:
            if line[i] == '':
                break
            else:
                row.append(line[i])
        myfile_writer.writerow(row)

__문제 발생__  
class에서 중복 발견  
=> line by line 중복 처리 필요 

In [37]:
csv_patent1 = open('veh_pid_class.csv','rb')
csv_patent_reader1 = csv.reader(csv_patent1)
header=csv_patent_reader1.next()

In [38]:
with open('veh_pid_class_no_dup.csv','wb') as myfile:
    myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    myfile_writer.writerow(header)
    for line in csv_patent_reader1:
        no_dup = list(set(line[2:]))
        row = line[0:2] + no_dup
        myfile_writer.writerow(row)

In [2]:
df=pd.read_csv('veh_pid_class_no_dup.csv')

C:\Users\DW\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


column 몇 개로 해야하는지 확인좀 (현재 20개 이하인건 확실)

In [11]:
check = df.iloc[:,20:22]
check.iloc[0:3,]

,class_19,class_20
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [16]:
len(check.index)*2

2878896

In [15]:
check.isnull().sum().sum()

2878895

오오 19에서 null 아닌 친구가 있나봄, 확인 ㄱㄱ

In [17]:
cl19 = check.iloc[:,0]
cl20 = check.iloc[:,1]
print len(cl19), cl19.isnull().sum(), cl20.isnull().sum()

1439448 1439447 1439448


good~ class 개수는 __19개__가 적당하군

### 1.2 main, sub 다 뽑기

In [1]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent")

In [2]:
zf_patent1 = zipfile.ZipFile('harvard_vehicle_patent1.zip') 
csv_patent1 = open('harvard_vehicle_patent1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header=csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('harvard_vehicle_patent2.zip') 
csv_patent2 = zf_patent2.open('harvard_vehicle_patent2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)
header=csv_patent_reader2.next()

In [3]:
class_col = []
for j in range(62457,63354):
    if 'class' in header[j]:   
        class_col.append(j)

In [5]:
head

['row',
 'patent_id',
 'mainclass_id_0',
 'subclass_id_0',
 'mainclass_id_1',
 'subclass_id_1',
 'mainclass_id_2',
 'subclass_id_2',
 'mainclass_id_3',
 'subclass_id_3',
 'mainclass_id_4',
 'subclass_id_4',
 'mainclass_id_5',
 'subclass_id_5',
 'mainclass_id_6',
 'subclass_id_6',
 'mainclass_id_7',
 'subclass_id_7',
 'mainclass_id_8',
 'subclass_id_8',
 'mainclass_id_9',
 'subclass_id_9',
 'mainclass_id_10',
 'subclass_id_10',
 'mainclass_id_11',
 'subclass_id_11',
 'mainclass_id_12',
 'subclass_id_12',
 'mainclass_id_13',
 'subclass_id_13',
 'mainclass_id_14',
 'subclass_id_14',
 'mainclass_id_15',
 'subclass_id_15',
 'mainclass_id_16',
 'subclass_id_16',
 'mainclass_id_17',
 'subclass_id_17',
 'mainclass_id_18',
 'subclass_id_18',
 'mainclass_id_19',
 'subclass_id_19',
 'mainclass_id_20',
 'subclass_id_20',
 'mainclass_id_21',
 'subclass_id_21',
 'mainclass_id_22',
 'subclass_id_22',
 'mainclass_id_23',
 'subclass_id_23',
 'mainclass_id_24',
 'subclass_id_24',
 'mainclass_id_25',
 's

In [4]:
head = ['row','patent_id']
head = head + list(header[i] for i in class_col)

In [5]:
with open('veh_pid_class.csv','wb') as myfile:
    myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    myfile_writer.writerow(head)
    
    for line in csv_patent_reader1:
        row = []
        row.append(line[0])
        row.append(line[1])
        for i in class_col:
            if line[i] == '':
                break
            elif 'class' in line[i]:
                break
            else:
                row.append(line[i])
        myfile_writer.writerow(row)
        
    for line in csv_patent_reader2:
        row = []
        row.append(line[0])
        row.append(line[1])
        for i in class_col:
            if line[i] == '':
                break
            else:
                row.append(line[i])
        myfile_writer.writerow(row)

In [6]:
with open('veh_pid_class.csv','rb') as csvfile:
    csv_reader = csv.reader(csvfile)
    maxlen = 0
    for line in csv_reader:
        a=len(line)
        if a > maxlen:
            maxlen = a
            
maxlen

362

subclass에 mainclass 정보까지 불필요하게 있음  
__=> mainclass 부분 제거 후 int로 만들자__

In [2]:
csvin = open('veh_pid_class.csv','rb')
csv_reader = csv.reader(csvin)
header = csv_reader.next()

In [3]:
subclass_col = []
for i in range(0,len(header)):
    if 'subclass' in header[i]:   
        subclass_col.append(i)

In [4]:
def to_int(row):    
    try:
        row=map(int,row)
    except ValueError:
        try:
            for i in range(0,len(row)):
                row[i]=int(row[i])
        except ValueError:
            pass  
        
    return row

In [5]:
with open('veh_pid_class_processed.csv','wb') as csvout:    
    csv_writer = csv.writer(csvout,quoting=csv.QUOTE_ALL)
    csv_writer.writerow(header)
    for line in csv_reader:
        if int(line[0]) > 2964076 :
            for i in subclass_col:
                try:
                    if line[i]=='':
                        break
                    else:
                        line[i]=line[i].split('/')[1]
                except IndexError:
                    break
            csv_writer.writerow(to_int(line))
            
        else:
            csv_writer.writerow(to_int(line))

### 2. citation edge list + date

patent_id, citation_id, date 해서 쭈루루룩 나열

In [1]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent")

In [3]:
zf_patent1 = zipfile.ZipFile('harvard_vehicle_patent1.zip') 
csv_patent1 = open('harvard_vehicle_patent1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header=csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('harvard_vehicle_patent2.zip') 
csv_patent2 = zf_patent2.open('harvard_vehicle_patent2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)
header1=csv_patent_reader2.next()

In [5]:
citation_col = []
for j in range(20377,62457):
    if 'citation' in header[j]:   
        citation_col.append(j)

In [7]:
with open('veh_pid_citation_duplicated.csv','wb') as myfile:
    myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)    
    for line in csv_patent_reader1:
        for i in citation_col:
            if line[i] == '':
                break
            else:
                try:
                    row = []
                    row.append(int(line[1]))
                    row.append(int(line[i]))
                    row.append(line[i+1])
                    myfile_writer.writerow(row)
                
                except ValueError:
                    pass
        
    for line in csv_patent_reader2:
        for i in citation_col:
            if line[i] == '':
                break
            else:
                try:
                    row = []
                    row.append(int(line[1]))
                    row.append(int(line[i]))
                    row.append(line[i+1])
                    myfile_writer.writerow(row)
                
                except ValueError:
                    pass

In [2]:
#remove duplicates
df=pd.read_csv('veh_pid_citation_duplicated.csv')
df=df.drop_duplicates()
df.to_csv('veh_pid_citation_no_dup.csv',index=False)

### 2.1 citation edge list 만들기

In [2]:
df=pd.read_csv('veh_pid_citation_no_dup.csv')
df = df.iloc[:,0:2]
list(df)

['patent_id', 'cited_id']

In [3]:
len(df)

17694187

In [4]:
df.drop_duplicates(inplace=True)
len(df)

17694022

In [5]:
df.to_csv('veh_citation_network.csv',index=False)

### 2.2 patent date 뽑기

In [4]:
df=pd.read_csv('veh_pid_citation_no_dup.csv')
list(df)

['patent_id', 'cited_id', 'granted_date']

In [15]:
df = df.iloc[:,1:3]
df.rename(columns={'cited_id': 'patent_id'}, inplace=True)
list(df)

['patent_id', 'granted_date']

In [22]:
pid = pd.read_csv('veh_pid_class_no_dup.csv')
pid = pid.iloc[:,1]
pid[0:3,]

0    4489382
1    4489383
2    4489384
Name: patent_id, dtype: object

In [24]:
pid = pid.to_frame()
type(pid)

pandas.core.frame.DataFrame

In [60]:
pid_date = pid.merge(df,how='left',on='patent_id',sort=True)
pid_date.iloc[0:30,]

,patent_id,granted_date
0,3858256,0000-00-00
1,3858256,1975-01-01
2,3858256,1975-01-01
3,3858266,0000-00-00
4,3858266,1975-01-01
5,3858267,0000-00-00
6,3858267,0000-00-00
7,3858267,0000-00-00
8,3858267,0000-00-00
9,3858267,0000-00-00


In [61]:
pid_date['granted_date'].replace({'0000-00-00': np.nan}, inplace=True)
pid_date.iloc[0:30,]

,patent_id,granted_date
0,3858256,NaN
1,3858256,1975-01-01
2,3858256,1975-01-01
3,3858266,NaN
4,3858266,1975-01-01
5,3858267,NaN
6,3858267,NaN
7,3858267,NaN
8,3858267,NaN
9,3858267,NaN


In [62]:
pid_date = pid_date.drop_duplicates()
pid_date.dropna(axis=0, inplace=True)
print len(pid_date), len(pid)

994886 1439448


In [72]:
final = pid.merge(pid_date,how='left',on='patent_id',sort=True)
print len(final),len(pid)

1446361 1439448


In [88]:
final.isnull().sum()

patent_id            0
granted_date    451475
dtype: int64

1. __date가 없는 patent__가 45만개
2. __같은 patent_id, 다른 granted_date__가 7000개

In [73]:
for i in range(0,len(final)):
    if final.iloc[i,0] == final.iloc[i+1,0]:
        print i
        break

1484


In [78]:
final.iloc[1484:1488,]

,patent_id,granted_date
1484,3865095,1975-02-01
1485,3865095,1999-09-01
1486,3865100,1975-02-01
1487,3865101,1975-02-01


모르겠다 일단 저장해놓자

In [89]:
final.to_csv('veh_pid_date.csv',index=False)

### 2.3 새 raw data 에서 date 뽑기

In [1]:
import csv
import os

os.chdir("E:/patent")
tsv = open('all_patent/grant_patent.txt','rb')
tsv_reader = csv.reader(tsv, delimiter='\t')
header=tsv_reader.next()

In [2]:
header

['id',
 'type',
 'number',
 'country',
 'date',
 'abstract',
 'title',
 'kind',
 'num_claims']

In [3]:
with open('veh_pid_date.csv', 'wb') as csvout:
    csv_writer = csv.writer(csvout)
    csv_writer.writerow(list(header[i] for i in [0,4]))

    for line in tsv_reader:
        csv_writer.writerow(list(line[i] for i in [0,4]))

In [8]:
import pandas as pd

df = pd.read_csv('patent_date.csv')

pid = pd.read_csv('veh_pid_class_no_dup.csv')
pid = pid.iloc[:,1]
pid = pid.to_frame()

In [9]:
pid_date = pid.merge(df,how='left',on='patent_id',sort=True)
pid_date.iloc[0:10,]

,patent_id,date
0,3858256,1975-01-07
1,3858266,1975-01-07
2,3858267,1975-01-07
3,3858280,1975-01-07
4,3858287,1975-01-07
5,3858289,1975-01-07
6,3858307,1975-01-07
7,3858308,1975-01-07
8,3858309,1975-01-07
9,3858320,1975-01-07


In [10]:
print len(pid), len(pid_date)

1439448 1439448


In [11]:
pid_date.to_csv('veh_pid_date.csv', index=False)